In [1]:
using TaylorSeries, TaylorIntegration
using Plots, LaTeXStrings
# plotly()
pyplot(grid = :false, legend = :false, size=(600,600), color=:black, markersize = 0.3 )
const Z = 2.0
const order = 20
const t0 = 0.0
const t_max = 0.5 
const abs_tol = 1.0E-20
const steps = 500_000;

In [2]:
#Regularización KS
f(x, y) = [x^2 - y^2, 2x*y]
f(v) = f(v...)

f (generic function with 2 methods)

In [3]:
function undrivenHelium2D!{T<:Number}(τ, q::Array{T,1}, dq::Array{T,1})
    Q₁x, Q₁y, Q₂x, Q₂y, P₁x, P₁y, P₂x, P₂y = q
    
    #Cantidades auxiliares
    Q₁² = Q₁x^2 + Q₁y^2
    Q₂² = Q₂x^2 + Q₂y^2
    P₁² = P₁x^2 + P₁y^2
    P₂² = P₂x^2 + P₂y^2
    t = Q₁²*Q₂²
    rf = f(Q₁x, Q₁y) - f(Q₂x, Q₂y)
    f₁, f₂ = rf
    fs = f₁^2 + f₂^2
    nf = sqrt(fs)
    c1 = 1 + 1/nf
    nf³ = nf^3
    factor1 = t/nf³
    s1 = -0.125P₂² + Z - Q₂²*c1
    s2 = factor1*f₁
    s3 = -0.125P₁² + Z - Q₁²*c1
    #@show c1
    dq[1], dq[2] = 0.25*Q₂²*[P₁x, P₁y]
    dq[3], dq[4] = 0.25*Q₁²*[P₂x, P₂y]
    dq[5] = 2*Q₁x*(s1 + s2) + factor1*f₂*Q₁y
    dq[6] = 2*Q₁y*(s1 - s2) + factor1*f₂*Q₁x
    dq[7] = 2*Q₂x*(s3 - s2) - factor1*f₂*Q₂y
    dq[8] = 2*Q₂y*(s3 + s2) - factor1*f₂*Q₂x
    nothing
end



var1 = set_variables("q₁x q₁y q₂x q₂y p₁x p₁y p₂x p₂y", order = 1)

function condini1D{T<:Number}(x10::T, px10::T)
    @assert x10 != 0
    Q1 = sqrt(x10)
    Q2 = 0.0
    P1 = 2*px10*sqrt(x10)    
    P2 = sqrt(8Z)    
    return [Q1, Q2, P1, P2]
end


function condini2D{T<:Number}(q₁x::T, q₁y::T, p₁x::T, p₁y::T, P₂y::T)
    @assert (q₁x > 0 || q₁y > 0) && P₂y^2 <= 8Z
    Q₁x = sqrt(sqrt(4q₁x^2 + q₁y^2) + 2q₁x)/2
    Q₁y = q₁y/(2Q₁x)
    Q₂x = 0.0
    Q₂y = 0.0
    P₁x = 2(Q₁x*p₁x + Q₁y*p₁y)
    P₁y = 2(Q₁x*p₁y - Q₁y*p₁x)
    P₂x = sqrt(8Z - P₂y^2)
    return T[Q₁x, Q₁y, Q₂x, Q₂y, P₁x, P₁y, P₂x, P₂y]
end




function regHam(q₁x, q₁y, q₂x, q₂y, p₁x, p₁y, p₂x, p₂y)
    #Cantidades auxiliares
    Q₁² = q₁x^2 + q₁y^2
    Q₂² = q₂x^2 + q₂y^2
    P₁² = p₁x^2 + p₁y^2
    P₂² = p₂x^2 + p₂y^2
    t = Q₁²*Q₂²
    rf = f(q₁x, q₁y) - f(q₂x, q₂y)
    f₁, f₂ = rf
    nf = (f₁^2 + f₂^2)^(1/2)
    
    H = 0.125*(P₁²*Q₂² + P₂²*Q₁²) - Z*(Q₁² + Q₂²) + t*(1.0 + 1.0/nf)
    return H
end


regHam(v) = regHam(v...)

regHam (generic function with 2 methods)

In [19]:
function var_jetcoeffs!{T<:Number}(eqsdiff!, t0::T, x::Vector{Taylor1{T}},
        dx::Vector{Taylor1{T}}, xaux::Vector{Taylor1{T}}, jx::Vector{Taylor1{T}},
        jdx::Vector{Taylor1{T}}, jxaux::Vector{Taylor1{T}},
        δx::Array{TaylorN{Taylor1{T}},1}, dδx::Array{TaylorN{Taylor1{T}},1},
        jac::Array{Taylor1{T},2}, vT::Vector{T})

    order = x[1].order
    vT[1] = t0

    # Dimensions of phase-space: dof
    dof = length(x)

    for ord in 1:order
        ordnext = ord+1

        # Set `xaux`, auxiliary vector of Taylor1 to order `ord`
        for j in eachindex(x)
            @inbounds xaux[j] = Taylor1( x[j].coeffs[1:ord] )
        end
        
        #Set `jxaux` 
        for j in eachindex(jx)
            @inbounds jxaux[j] = Taylor1( jx[j].coeffs[1:ord] ) 
        end
        
        # Equations of motion
        tT = Taylor1(vT[1:ord])
        eqsdiff!(tT, xaux, dx)
        
        # stabilitymatrix!( eqsdiff!, t0, xaux[1:dof], δx, dδx, jac )
        TaylorIntegration.stabilitymatrix!( eqsdiff!, t0, xaux, δx, dδx, jac )
        @inbounds jdx[1:end] = jac * reshape( jxaux[1:end], (dof,dof) )
        
        # Recursion relations
        for j in eachindex(x)
            @inbounds x[j][ordnext] = dx[j][ord]/ord
        end
        for j in eachindex(jx)
            @inbounds jx[j][ordnext] = jdx[j][ord]/ord
        end
        
    end
    nothing
end

function var_taylorstep!{T<:Number}(f!, x::Vector{Taylor1{T}}, dx::Vector{Taylor1{T}},
        xaux::Vector{Taylor1{T}}, jx::Vector{Taylor1{T}}, jdx::Vector{Taylor1{T}},
        jxaux::Vector{Taylor1{T}}, δx::Array{TaylorN{Taylor1{T}},1},
    dδx::Array{TaylorN{Taylor1{T}},1}, jac::Array{Taylor1{T},2}, t0::T, t1::T, x0::Array{T,1}, 
        arrjac::Array{T,1}, order::Int, abstol::T, vT::Vector{T})
    
    # Compute the Taylor coefficients
    var_jetcoeffs!(f!, t0, x, dx, xaux, jx, jdx, jxaux, δx, dδx, jac, vT)
    
    # Compute the step-size of the integration using `abstol`
    δt = TaylorIntegration.stepsize(x, abstol)
    δt = min(δt, t1-t0)
    
    
    # Update x0
    evaluate!(x, δt, x0)
    
    #Update jt
    evaluate!(jx, δt, arrjac)
    
    return δt
end


var_taylorstep! (generic function with 2 methods)

In [20]:
#Versión de ecuaciones variacionales
function vartaylorinteg{T<:Number}(f!, q0::Array{T,1}, t0::T, tmax::T,
        order::Int, abstol::T; maxsteps::Int=500)

    # Allocation
    const tv = Array{T}(maxsteps+1)
    dof = length(q0)
    const xv = Array{T}(dof, maxsteps+1)
    const jt = eye(T, dof, dof)
    const mv = Array{Array{T,2}}(maxsteps+1)
    const vT = zeros(T, order+1)
    vT[2] = one(T)

    
    # NOTE: This changes GLOBALLY internal parameters of TaylorN
    global _δv = set_variables("δ", order=1, numvars=dof)
    
    # Initialize the vector of Taylor1 expansions
    const x = Array{Taylor1{T}}(dof)
    const dx = Array{Taylor1{T}}(dof)
    const xaux = Array{Taylor1{T}}(dof)
    
    #Taylor1 for jacobian
    const jx = Array{Taylor1{T}}(dof*dof)
    const jdx = Array{Taylor1{T}}(dof*dof)
    const jxaux = Array{Taylor1{T}}(dof*dof)
    const δx = Array{TaylorN{Taylor1{T}}}(dof)
    const dδx = Array{TaylorN{Taylor1{T}}}(dof)
    const jac = Array{Taylor1{T}}(dof, dof)
    const arrjac = Array{T}(dof*dof)
    for i in eachindex(q0)
        @inbounds x[i] = Taylor1( q0[i], order )
    end
    
    #auxiliary arrays for symplectic structure tests
    const δSv = Array{T}(maxsteps+1); δSv[1] = zero(T)
    auxJn = Int(dof/2)
    const J_n = vcat(  hcat(zeros(auxJn,auxJn), eye(auxJn,auxJn)), hcat(-eye(auxJn,auxJn), zeros(auxJn,auxJn))  )

    # Initial conditions
    @inbounds tv[1] = t0
    @inbounds xv[:,1] .= q0
    @inbounds mv[1] = jt
    x0 = copy(q0)
    arrjac = reshape(jt, dof*dof)
    
    for i in eachindex(arrjac)
        @inbounds jx[i] = Taylor1( arrjac[i], order )
    end
    
    # Integration
    nsteps = 1
    while t0 < tmax
        δt = var_taylorstep!(f!, x, dx, xaux, jx, jdx, jxaux, δx, dδx, jac, t0, tmax, x0, 
        arrjac, order, abstol, vT)
        
        #Taking stability matrix values
        for ind in eachindex(jt)
            @inbounds jt[ind] = arrjac[ind]
        end
        
        t0 += δt
        nsteps += 1
        @inbounds tv[nsteps] = t0
        
        
        
        δSv[nsteps] = norm( jt'*J_n*jt-J_n, Inf)
        
        for i in eachindex(x0)
            @inbounds x[i][1] = x0[i]
        end
        
        for i in eachindex(arrjac)
            @inbounds jx[i][1] = arrjac[i]
        end

        @inbounds xv[:,nsteps] .= x0
        if nsteps > maxsteps
            warn("""
            Maximum number of integration steps reached; exiting.
            """)
            break
        end
    end

    return view(tv,1:nsteps), view(transpose(xv),1:nsteps,:), δSv, mv
end

vartaylorinteg (generic function with 1 method)

In [7]:
#Initial conditions
c1 = condini2D(2.577, 0.0, 0.0, 0.0, 0.0)

8-element Array{Float64,1}:
 1.6053
 0.0   
 0.0   
 0.0   
 0.0   
 0.0   
 4.0   
 0.0   

In [24]:
@time tvS, xvS, δSv, mv = vartaylorinteg(undrivenHelium2D!, c1, 0.0, 30.0, 25, 1e-20, maxsteps=30);  

  0.912753 seconds (15.03 M allocations: 1.084 GiB, 36.51% gc time)


In [25]:
norm(δSv,Inf)

4.388036509840388e-15

In [26]:
xvS

31×8 SubArray{Float64,2,Array{Float64,2},Tuple{UnitRange{Int64},Base.Slice{Base.OneTo{Int64}}},false}:
 1.6053   0.0   0.0       0.0   0.0         0.0   4.0       0.0
 1.6053   0.0   0.277516  0.0  -0.00334061  0.0   3.95166   0.0
 1.60527  0.0   0.497949  0.0  -0.0177552   0.0   3.8346    0.0
 1.60507  0.0   0.680534  0.0  -0.0389789   0.0   3.66301   0.0
 1.60464  0.0   0.82576   0.0  -0.0539923   0.0   3.44958   0.0
 1.60403  0.0   0.949219  0.0  -0.0481204   0.0   3.17534   0.0
 1.60366  0.0   1.0402    0.0  -0.00965243  0.0   2.87488   0.0
 1.60396  0.0   1.11069   0.0   0.0679335   0.0   2.53591   0.0
 1.60541  0.0   1.16504   0.0   0.188354    0.0   2.15605   0.0
 1.60844  0.0   1.20591   0.0   0.350296    0.0   1.73834   0.0
 1.61342  0.0   1.23535   0.0   0.547869    0.0   1.28838   0.0
 1.62071  0.0   1.25496   0.0   0.773371    0.0   0.809965  0.0
 1.63121  0.0   1.26562   0.0   1.02657     0.0   0.28787   0.0
 ⋮                                          ⋮                  
 

In [ ]:
regHam(xvS[end,:])

In [ ]:
xvS

In [ ]:
#Poincare functions
function poincare{T<:Number}(f!, g, q0::Array{T,1}, t0::T, tmax::T,
        order::Int, abstol::T; maxsteps::Int=500, nriter::Int=5)

    # Allocation
    const tv = Array{T}(maxsteps+1)
    dof = length(q0)
    const xv = Array{T}(dof, maxsteps+1)
    const vT = zeros(T, order+1)
    vT[2] = one(T)

    # Initialize the vector of Taylor1 expansions
    const x = Array{Taylor1{T}}(dof)
    const dx = Array{Taylor1{T}}(dof)
    const xaux = Array{Taylor1{T}}(dof)
    for i in eachindex(q0)
        @inbounds x[i] = Taylor1( q0[i], order )
    end

    # Initial conditions
    @inbounds tv[1] = t0
    @inbounds xv[:,1] .= q0
    x0 = copy(q0)

    #Some auxiliary arrays for root-finding/event detection/Poincaré surface of section evaluation
    const g_val = Taylor1(zero(T), order)
    const g_val_old = Taylor1(zero(T), order)
    const slope = zero(T)
    const dt_li = zero(T)
    const dt_nr = zero(T)
    const δt = zero(T)
    const δt_old = zero(T)

    const x_g_Dg_D2g = vcat(x, dx, zero(x[1]), zero(x[1]))
    const x_g_Dg_D2g_val = Array{T}( length(x_g_Dg_D2g) )

    const tvS = similar(tv)
    const xvS = similar(xv)
    const gvS = similar(tv)

    #auxiliary range object for Newton-Raphson iterations
    const nrinds = 1:nriter

    # Integration
    nsteps = 1
    nevents = 1 #number of detected events
    while t0 < tmax
        δt_old = δt
        δt = TaylorIntegration.taylorstep!(f!, x, dx, xaux, t0, tmax, x0, order, abstol, vT)
        g_val = g(t0, x, dx)
        if g_val_old[1]*g_val[1] < zero(T)

            #first guess: linear interpolation
            slope = (g_val[1]-g_val_old[1])/δt_old
            dt_li = -(g_val[1]/slope)

            x_g_Dg_D2g[1:dof] = x
            x_g_Dg_D2g[dof+1:2dof] = dx
            x_g_Dg_D2g[2dof+1] = g_val
            x_g_Dg_D2g[2dof+2] = derivative(g_val)

            #Newton-Raphson iterations
            dt_nr = dt_li
            evaluate!(x_g_Dg_D2g[2dof+1:2dof+2], dt_nr, view(x_g_Dg_D2g_val,2dof+1:2dof+2))

            for i in eachindex(nrinds)
                dt_nr = dt_nr-x_g_Dg_D2g_val[2dof+1]/x_g_Dg_D2g_val[2dof+2]
                evaluate!(x_g_Dg_D2g[2dof+1:2dof+2], dt_nr, view(x_g_Dg_D2g_val,2dof+1:2dof+2))
            end
            evaluate!(x_g_Dg_D2g[1:2dof], dt_nr, view(x_g_Dg_D2g_val,1:2dof))

            tvS[nevents] = t0+dt_nr
            xvS[:,nevents] .= view(x_g_Dg_D2g_val,1:dof)
            gvS[nevents] = x_g_Dg_D2g_val[2dof+1]

            nevents += 1
        end
        g_val_old = deepcopy(g_val)
        for i in eachindex(x0)
            @inbounds x[i][1] = x0[i]
        end
        t0 += δt
        nsteps += 1
        @inbounds tv[nsteps] = t0
        @inbounds xv[:,nsteps] .= x0
        if nsteps > maxsteps
            warn("""
            Maximum number of integration steps reached; exiting.
            """)
            break
        end
    end

    return view(tv,1:nsteps), view(transpose(xv),1:nsteps,:), view(tvS,1:nevents-1), view(transpose(xvS),1:nevents-1,:), view(gvS,1:nevents-1)
end

#poincare + jet transport
function poincare{T<:Number}(f!, g, q0::Array{TaylorN{T},1}, t0::T, tmax::T,
        order::Int, abstol::T; maxsteps::Int=500, nriter::Int=5)

    # Allocation
    const tv = Array{T}(maxsteps+1)
    dof = length(q0)
    const xv = Array{TaylorN{T}}(dof, maxsteps+1)
    const vT = zeros(T, order+1)
    vT[2] = one(T)

    # Initialize the vector of Taylor1 expansions
    const x = Array{Taylor1{TaylorN{T}}}(dof)
    const dx = Array{Taylor1{TaylorN{T}}}(dof)
    const xaux = Array{Taylor1{TaylorN{T}}}(dof)
    for i in eachindex(q0)
        @inbounds x[i] = Taylor1( q0[i], order )
    end

    # Initial conditions
    @inbounds tv[1] = t0
    @inbounds xv[:,1] .= q0
    x0 = copy(q0)

    #Some auxiliary arrays for root-finding/event detection/Poincaré surface of section evaluation
    zeroTN = zero(q0[1])
    const g_val = Taylor1(zeroTN, order)
    const g_val_old = Taylor1(zeroTN, order)
    const slope = zeroTN
    const dt_li = zeroTN
    const dt_nr = zeroTN
    const δt = zeroTN
    const δt_old = zeroTN

    const x_g_Dg_D2g = vcat(x, dx, zero(x[1]), zero(x[1]))
    const x_g_Dg_D2g_val = Array{TaylorN{T}}( length(x_g_Dg_D2g) )

    const tvS = Array{TaylorN{T}}( length(tv) )
    const xvS = similar(xv)
    const gvS = similar(tvS)

    #auxiliary range object for Newton-Raphson iterations
    const nrinds = 1:nriter

    # Integration
    nsteps = 1
    nevents = 1 #number of detected events
    while t0 < tmax
        δt_old = δt
        δt = TaylorIntegration.taylorstep!(f!, x, dx, xaux, t0, tmax, x0, order, abstol, vT)
        g_val = g(t0, x, dx)
        if g_val_old[1][1][1]*g_val[1][1][1] < zero(T)

            #first guess: linear interpolation
            slope = (g_val[1]-g_val_old[1])/δt_old
            dt_li = -(g_val[1]/slope)

            x_g_Dg_D2g[1:dof] = x
            x_g_Dg_D2g[dof+1:2dof] = dx
            x_g_Dg_D2g[2dof+1] = g_val
            x_g_Dg_D2g[2dof+2] = derivative(g_val)

            #Newton-Raphson iterations
            dt_nr = dt_li
            evaluate!(x_g_Dg_D2g[2dof+1:2dof+2], dt_nr, view(x_g_Dg_D2g_val,2dof+1:2dof+2))

            for i in eachindex(nrinds)
                dt_nr = dt_nr-x_g_Dg_D2g_val[2dof+1]/x_g_Dg_D2g_val[2dof+2]
                evaluate!(x_g_Dg_D2g[2dof+1:2dof+2], dt_nr, view(x_g_Dg_D2g_val,2dof+1:2dof+2))
            end
            evaluate!(x_g_Dg_D2g[1:2dof], dt_nr, view(x_g_Dg_D2g_val,1:2dof))

            tvS[nevents] = t0+dt_nr
            xvS[:,nevents] .= view(x_g_Dg_D2g_val,1:dof)
            gvS[nevents] = x_g_Dg_D2g_val[2dof+1]

            nevents += 1
        end
        g_val_old = deepcopy(g_val)
        for i in eachindex(x0)
            @inbounds x[i][1] = x0[i]
        end
        t0 += δt
        nsteps += 1
        @inbounds tv[nsteps] = t0
        @inbounds xv[:,nsteps] .= x0
        if nsteps > maxsteps
            warn("""
            Maximum number of integration steps reached; exiting.
            """)
            break
        end
    end

    return view(tv,1:nsteps), transpose(view(xv,:,1:nsteps)), view(tvS,1:nevents-1), transpose(view(xvS,:,1:nevents-1)), view(gvS,1:nevents-1)
end

function poincare2{T<:Number}(f!, g, q0::Array{T,1}, t0::T, tmax::T,
        order::Int, abstol::T; maxsteps::Int=500, nriter::Int=5)

    # Allocation
    const tv = Array{T}(maxsteps+1)
    dof = length(q0)
    const xv = Array{T}(dof, maxsteps+1)
    const vT = zeros(T, order+1)
    vT[2] = one(T)

    # Initialize the vector of Taylor1 expansions
    const x = Array{Taylor1{T}}(dof)
    const dx = Array{Taylor1{T}}(dof)
    const xaux = Array{Taylor1{T}}(dof)
    for i in eachindex(q0)
        @inbounds x[i] = Taylor1( q0[i], order )
    end

    # Initial conditions
    @inbounds tv[1] = t0
    @inbounds xv[:,1] .= q0
    x0 = copy(q0)

    #Some auxiliary arrays for root-finding/event detection/Poincaré surface of section evaluation
    const g_val = Taylor1(zero(T), order)
    const g_val_old = Taylor1(zero(T), order)
    const slope = zero(T)
    const dt_li = zero(T)
    const dt_nr = zero(T)
    const δt = zero(T)
    const δt_old = zero(T)

    const x_g_Dg_D2g = vcat(x, dx, zero(x[1]), zero(x[1]))
    const x_g_Dg_D2g_val = Array{T}( length(x_g_Dg_D2g) )

    const tvS = similar(tv)
    const xvS = similar(xv)
    const gvS = similar(tv)

    #auxiliary range object for Newton-Raphson iterations
    const nrinds = 1:nriter

    # Integration
    nsteps = 1
    nevents = 1 #number of detected events
    while t0 < tmax
        δt_old = δt
        δt = TaylorIntegration.taylorstep!(f!, x, dx, xaux, t0, tmax, x0, order, abstol, vT)
        g_val = g(t0, x, dx)
        if g_val_old[2]*g_val[2] < zero(T)

            #first guess: linear interpolation
            slope = (g_val[2]-g_val_old[2])/δt_old
            dt_li = -(g_val[2]/slope)

            x_g_Dg_D2g[1:dof] = x
            x_g_Dg_D2g[dof+1:2dof] = dx
            x_g_Dg_D2g[2dof+1] = derivative(g_val)
            x_g_Dg_D2g[2dof+2] = derivative(x_g_Dg_D2g[2dof+1])

            #Newton-Raphson iterations
            dt_nr = dt_li
            evaluate!(x_g_Dg_D2g[2dof+1:2dof+2], dt_nr, view(x_g_Dg_D2g_val,2dof+1:2dof+2))

            for i in eachindex(nrinds)
                dt_nr = dt_nr-x_g_Dg_D2g_val[2dof+1]/x_g_Dg_D2g_val[2dof+2]
                evaluate!(x_g_Dg_D2g[2dof+1:2dof+2], dt_nr, view(x_g_Dg_D2g_val,2dof+1:2dof+2))
            end
            evaluate!(x_g_Dg_D2g[1:2dof], dt_nr, view(x_g_Dg_D2g_val,1:2dof))

            tvS[nevents] = t0+dt_nr
            xvS[:,nevents] .= view(x_g_Dg_D2g_val,1:dof)
            gvS[nevents] = x_g_Dg_D2g_val[2dof+1]

            nevents += 1
        end
        g_val_old = deepcopy(g_val)
        for i in eachindex(x0)
            @inbounds x[i][1] = x0[i]
        end
        t0 += δt
        nsteps += 1
        @inbounds tv[nsteps] = t0
        @inbounds xv[:,nsteps] .= x0
        if nsteps > maxsteps
            warn("""
            Maximum number of integration steps reached; exiting.
            """)
            break
        end
    end

    return view(tv,1:nsteps), view(transpose(xv),1:nsteps,:), view(tvS,1:nevents-1), view(transpose(xvS),1:nevents-1,:), view(gvS,1:nevents-1)
end

#poincare2 + jet transport
function poincare2{T<:Number}(f!, g, q0::Array{TaylorN{T},1}, t0::T, tmax::T,
        order::Int, abstol::T; maxsteps::Int=500, nriter::Int=5)

    # Allocation
    const tv = Array{T}(maxsteps+1)
    dof = length(q0)
    const xv = Array{TaylorN{T}}(dof, maxsteps+1)
    const vT = zeros(T, order+1)
    vT[2] = one(T)

    # Initialize the vector of Taylor1 expansions
    const x = Array{Taylor1{TaylorN{T}}}(dof)
    const dx = Array{Taylor1{TaylorN{T}}}(dof)
    const xaux = Array{Taylor1{TaylorN{T}}}(dof)
    for i in eachindex(q0)
        @inbounds x[i] = Taylor1( q0[i], order )
    end

    # Initial conditions
    @inbounds tv[1] = t0
    @inbounds xv[:,1] .= q0
    x0 = copy(q0)

    #Some auxiliary arrays for root-finding/event detection/Poincaré surface of section evaluation
    zeroTN = zero(q0[1])
    const g_val = Taylor1(zeroTN, order)
    const g_val_old = Taylor1(zeroTN, order)
    const slope = zeroTN
    const dt_li = zeroTN
    const dt_nr = zeroTN
    const δt = zeroTN
    const δt_old = zeroTN

    const x_g_Dg_D2g = vcat(x, dx, zero(x[1]), zero(x[1]))
    const x_g_Dg_D2g_val = Array{TaylorN{T}}( length(x_g_Dg_D2g) )

    const tvS = Array{TaylorN{T}}( length(tv) )
    const xvS = similar(xv)
    const gvS = similar(tvS)

    #auxiliary range object for Newton-Raphson iterations
    const nrinds = 1:nriter

    # Integration
    nsteps = 1
    nevents = 1 #number of detected events
    while t0 < tmax
        δt_old = δt
        δt = TaylorIntegration.taylorstep!(f!, x, dx, xaux, t0, tmax, x0, order, abstol, vT)
        g_val = g(t0, x, dx)
        if g_val_old[2][1][1]*g_val[2][1][1] < zero(T)

            #first guess: linear interpolation
            slope = (g_val[2]-g_val_old[2])/δt_old
            dt_li = -(g_val[2]/slope)

            x_g_Dg_D2g[1:dof] = x
            x_g_Dg_D2g[dof+1:2dof] = dx
            x_g_Dg_D2g[2dof+1] = derivative(g_val)
            x_g_Dg_D2g[2dof+2] = derivative(x_g_Dg_D2g[2dof+1])

            #Newton-Raphson iterations
            dt_nr = dt_li
            evaluate!(x_g_Dg_D2g[2dof+1:2dof+2], dt_nr, view(x_g_Dg_D2g_val,2dof+1:2dof+2))

            for i in eachindex(nrinds)
                dt_nr = dt_nr-x_g_Dg_D2g_val[2dof+1]/x_g_Dg_D2g_val[2dof+2]
                evaluate!(x_g_Dg_D2g[2dof+1:2dof+2], dt_nr, view(x_g_Dg_D2g_val,2dof+1:2dof+2))
            end
            evaluate!(x_g_Dg_D2g[1:2dof], dt_nr, view(x_g_Dg_D2g_val,1:2dof))

            tvS[nevents] = t0+dt_nr
            xvS[:,nevents] .= view(x_g_Dg_D2g_val,1:dof)
            gvS[nevents] = x_g_Dg_D2g_val[2dof+1]

            nevents += 1
        end
        g_val_old = deepcopy(g_val)
        for i in eachindex(x0)
            @inbounds x[i][1] = x0[i]
        end
        t0 += δt
        nsteps += 1
        @inbounds tv[nsteps] = t0
        @inbounds xv[:,nsteps] .= x0
        if nsteps > maxsteps
            warn("""
            Maximum number of integration steps reached; exiting.
            """)
            break
        end
    end

    return view(tv,1:nsteps), view(transpose(xv),1:nsteps,:), view(tvS,1:nevents-1), view(transpose(xvS),1:nevents-1,:), view(gvS,1:nevents-1)
end

In [ ]:
x1[end,:]

In [ ]:
jacobian(x1[end,:])

In [ ]:
length(x1)

In [ ]:
#Ahora obtenemos los jacobianos y las posiciones a partir de los resultados de la integración
function separa{T<:Number}(qs::Array{TaylorN{T},1})
    #Allocations
    ll = length(qs)
    xs = Array{T}(ll)
    js = Array{Array{T,2}}(ll)
    
    #
    

In [ ]:
x1[end,:][1][2]

In [ ]:
typeof(jacobian(x1[end,:]))